In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# ---------- CONFIGURATION ----------

# Status paths and weights
STATUS_PATHS = {
    'a': ([1, 2, 3, 4], 70),
    'b': ([1, 2, 5], 3),
    'c': ([1, 2, 3, 6, 2, 3, 4], 7),
    'd': ([1, 2, 3, 6, 2, 3, 5], 5),
    'e': ([1, 2, 7], 7),
    'f': ([1, 2, 7, 3, 4], 8)
}
STATUS_ID_TO_NAME = {
    1: "Open", 2: "In Progress", 3: "Resolved",
    4: "Closed", 5: "Invalid", 6: "Escalated", 7: "Temp Resolved"
}

# For demonstration, you can set these filenames
INPUT_XLSX = 'tickets_input.xlsx'
OUTPUT_XLSX = 'ticket_history_output.xlsx'

# ---------- MODULAR TIMESTAMP LOGIC ----------

def generate_updated_at(raised_at, expiry_at, step, total_steps):
    """
    Generates an updated_at timestamp for each status change.
    Most steps will be within expiry_at, some can exceed by up to 2 days.
    """
    raised_dt = pd.to_datetime(raised_at)
    expiry_dt = pd.to_datetime(expiry_at)
    # Spread steps between raised_at and expiry_at (sometimes exceed)
    if random.random() < 0.8:
        # Within expiry
        delta = (expiry_dt - raised_dt) / total_steps
        updated_at = raised_dt + delta * step
    else:
        # Exceed by up to 2 days
        delta = (expiry_dt - raised_dt) / total_steps
        updated_at = expiry_dt + timedelta(days=random.uniform(0.1, 2))
    return updated_at

# ---------- ASSIGNEE AND UPDATED_BY LOGIC ----------

def get_new_assignee(current_assignee, employee_pool):
    """Randomly pick a new assignee different from the current one."""
    possible = [emp for emp in employee_pool if emp != current_assignee]
    return random.choice(possible) if possible else current_assignee

def get_updated_by(assigned_to, employee_pool):
    """Randomly pick who updated the ticket (could be assignee or someone else)."""
    if random.random() < 0.7:
        return assigned_to
    else:
        return random.choice(employee_pool)

# ---------- MAIN PROCESS ----------

def generate_ticket_history(input_xlsx, output_xlsx):
    # Read ticket data
    tickets = pd.read_excel(input_xlsx)
    # Prepare employee pool for assignment/updates
    all_employees = set(tickets['EMPLOYEE_ID_FK']).union(set(tickets['ASSIGNED_TO']))

    history_records = []

    for idx, ticket in tickets.iterrows():
        # Select a status path based on weights
        paths, weights = zip(*[(v[0], v[1]) for v in STATUS_PATHS.values()])
        chosen_path = random.choices(paths, weights=weights, k=1)[0]
        total_steps = len(chosen_path)
        assigned_to = ticket['ASSIGNED_TO']

        for step, status_id in enumerate(chosen_path):
            # Handle assignee change on escalation or temp resolved
            if status_id in [6, 7]:  # Escalated or Temp Resolved
                assigned_to = get_new_assignee(assigned_to, all_employees)
                updated_by = get_updated_by(assigned_to, all_employees)
            else:
                if status_id in [1]:  # Open
                    assigned_to = 0
                    updated_by = ticket['RAISED_BY']
                if status_id in [1]:  # InProgress
                    assigned_to = ticket['ASSIGNED_TO']
                    if random.random()<0.8:
                      updated_by = ticket['ASSIGNED_TO']
                    else:
                      updated_by = get_updated_by(assigned_to, all_employees)
                if status_id in [4]:  # Closed
                    assigned_to = ticket['ASSIGNED_TO']
                    updated_by = ticket['RAISED_BY']
                if status_id in [5]:  # Invalid
                    assigned_to = ticket['ASSIGNED_TO']
                    updated_by = ticket['ASSIGNED_TO']
                if status_id in [3]:  # Resolved
                    assigned_to = ticket['ASSIGNED_TO']
                    updated_by = ticket['ASSIGNED_TO']


            updated_at = generate_updated_at(ticket['RAISED_AT'], ticket['EXPIRY_AT'], step, total_steps)
            # Build history record (excluding subject/description)
            record = {
                'TICKET_ID': ticket['TICKET_ID'],
                'EMPLOYEE_ID_FK': ticket['EMPLOYEE_ID_FK'],
                'RAISED_AT': ticket['RAISED_AT'],
                'RAISED_BY': ticket['RAISED_BY'],
                'CATEGORY_ID': ticket['CATEGORY_ID'],
                'SUB_CATEGORY_ID': ticket['SUB_CATEGORY_ID'],
                'EXPIRY_AT': ticket['EXPIRY_AT'],
                'ASSIGNED_TO': assigned_to,
                'STATUS_ID': status_id,
                'STATUS_NAME': STATUS_ID_TO_NAME[status_id],
                'IS_ACTIVE': ticket['IS_ACTIVE'],
                'UPDATED_AT': updated_at,
                'UPDATED_BY': updated_by
            }
            # Optionally include UPDATED_AT and UPDATED_BY from ticket if needed
            history_records.append(record)

    # Output to Excel
    history_df = pd.DataFrame(history_records)
    history_df.sort_values(['TICKET_ID', 'UPDATED_AT'], inplace=True)
    history_df.to_excel(output_xlsx, index=False)

# ---------- RUN SCRIPT ----------

if __name__ == "__main__":
    generate_ticket_history(INPUT_XLSX, OUTPUT_XLSX)


In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import files

# ---------- CONFIGURATION ----------

# Status paths and weights
STATUS_PATHS = {
    'a': ([1, 2, 3, 4], 70),
    'b': ([1, 2, 5], 3),
    'c': ([1, 2, 3, 6, 2, 3, 4], 7),
    'd': ([1, 2, 3, 6, 2, 3, 5], 5),
    'e': ([1, 2, 7], 7),
    'f': ([1, 2, 7, 3, 4], 8)
}
STATUS_ID_TO_NAME = {
    1: "Open", 2: "In Progress", 3: "Resolved",
    4: "Closed", 5: "Invalid", 6: "Escalated", 7: "Temp Resolved"
}

# ---------- MODULAR TIMESTAMP LOGIC ----------

def generate_updated_at(raised_at, expiry_at, step, total_steps):
    """
    Generates updated_at timestamps for each status change step.
    - 80% chance timestamp lies between raised_at and expiry_at.
    - 20% chance timestamp lies between expiry_at and expiry_at + 2 days.
    - Timestamps strictly increase with step number.
    """
    raised_dt = pd.to_datetime(raised_at)
    expiry_dt = pd.to_datetime(expiry_at)
    max_dt = expiry_dt + timedelta(days=2)

    # Decide if this step exceeds expiry (20% chance)
    exceed_expiry = random.random() < 0.1

    if exceed_expiry:
        # Spread steps that exceed expiry evenly between expiry and max_dt
        exceed_duration = max_dt - expiry_dt
        base_interval = exceed_duration / total_steps
        base_time = expiry_dt + base_interval * step
        # Add small jitter (+/- 10% of base_interval)
        jitter_seconds = base_interval.total_seconds() * 0.1
        jitter = timedelta(seconds=random.uniform(-jitter_seconds, jitter_seconds))
        updated_at = base_time + jitter
        # Clamp within [expiry_dt, max_dt]
        updated_at = max(expiry_dt, min(updated_at, max_dt))
    else:
        # Spread steps within raised_at and expiry_at
        duration = expiry_dt - raised_dt
        base_interval = duration / total_steps
        base_time = raised_dt + base_interval * step
        jitter_seconds = base_interval.total_seconds() * 0.1
        jitter = timedelta(seconds=random.uniform(-jitter_seconds, jitter_seconds))
        updated_at = base_time + jitter
        # Clamp within [raised_dt, expiry_dt]
        updated_at = max(raised_dt, min(updated_at, expiry_dt))

    return updated_at


# ---------- ASSIGNEE AND UPDATED_BY LOGIC ----------

def get_new_assignee(current_assignee, employee_pool):
    possible = [emp for emp in employee_pool if emp != current_assignee]
    return random.choice(possible) if possible else current_assignee

def get_updated_by(assigned_to, employee_pool):
    if random.random() < 0.7:
        return assigned_to
    else:
        return random.choice(employee_pool)

# ---------- MAIN PROCESS ----------

def generate_ticket_history(tickets):
    all_employees = list(set(tickets['EMPLOYEE_ID_FK']).union(set(tickets['ASSIGNED_TO'])))
    history_records = []

    for idx, ticket in tickets.iterrows():
        paths, weights = zip(*[(v[0], v[1]) for v in STATUS_PATHS.values()])
        chosen_path = random.choices(paths, weights=weights, k=1)[0]
        total_steps = len(chosen_path)
        assigned_to = ticket['ASSIGNED_TO']

        for step, status_id in enumerate(chosen_path):
            if status_id == 1:  # Open
                assigned_to = 0  # Unassigned
                updated_by = ticket['RAISED_BY']
            elif status_id == 2:  # In Progress
                assigned_to = ticket['ASSIGNED_TO']
                updated_by = assigned_to if random.random() < 0.8 else get_updated_by(assigned_to, all_employees)
            elif status_id == 3:  # Resolved
                assigned_to = ticket['ASSIGNED_TO']
                updated_by = assigned_to
            elif status_id == 4:  # Closed
                assigned_to = ticket['ASSIGNED_TO']
                updated_by = ticket['RAISED_BY']
            elif status_id == 5:  # Invalid
                assigned_to = ticket['ASSIGNED_TO']
                updated_by = assigned_to
            elif status_id in [6, 7]:  # Escalated or Temp Resolved
                assigned_to = get_new_assignee(assigned_to, all_employees)
                updated_by = get_updated_by(assigned_to, all_employees)


            updated_at = generate_updated_at(ticket['RAISED_AT'], ticket['EXPIRY_AT'], step, total_steps)

            record = {
                'TICKET_ID': ticket['TICKET_ID '],
                'EMPLOYEE_ID_FK': ticket['EMPLOYEE_ID_FK'],
                'RAISED_AT': ticket['RAISED_AT'],
                'RAISED_BY': ticket['RAISED_BY'],
                'CATEGORY_ID': ticket['CATEGORY_ID'],
                'SUB_CATEGORY_ID': ticket['SUB_CATEGORY_ID'],
                'EXPIRY_AT': ticket['EXPIRY_AT'],
                'ASSIGNED_TO': assigned_to,
                'STATUS_ID': status_id,
                'STATUS_NAME': STATUS_ID_TO_NAME[status_id],
                'IS_ACTIVE': ticket['IS_ACTIVE'],
                'UPDATED_AT': updated_at,
                'UPDATED_BY': updated_by
            }
            history_records.append(record)

    history_df = pd.DataFrame(history_records)
    history_df.sort_values(['TICKET_ID', 'UPDATED_AT'], inplace=True)
    return history_df

# ---------- GOOGLE COLAB FILE UPLOAD & RUN ----------

def main():
    print("Please upload your input Excel file (e.g., tickets_input.xlsx):")
    #uploaded = files.upload("/content/Help desk data.xlsx")  # Upload widget

    # Get the uploaded filename
    #input_filename = list(uploaded.keys())[0]
    #print(f"Uploaded file: {input_filename}")

    # Read input Excel
    tickets = pd.read_excel("/content/Help desk data (1).xlsx",sheet_name = "Sheet1")

    # Generate history
    history_df = generate_ticket_history(tickets)

    # Save output Excel
    output_filename = 'ticket_history_output.xlsx'
    history_df.to_excel(output_filename, index=False)
    print(f"Ticket history generated and saved to {output_filename}")

    # Provide download link
    #files.download(output_filename)

if __name__ == "__main__":
    main()


Please upload your input Excel file (e.g., tickets_input.xlsx):
Ticket history generated and saved to ticket_history_output.xlsx


In [ ]:
import pandas as pd
import random
from datetime import timedelta
import math

def generate_updated_at(raised_at, expiry_at, step, total_steps):
    """
    Generates strictly increasing timestamps for each step,
    ensuring all timestamps fall between raised_at and expiry_at.
    """
    raised_dt = pd.to_datetime(raised_at)
    expiry_dt = pd.to_datetime(expiry_at)

    # Calculate the total duration for status changes
    total_duration = expiry_dt - raised_dt

    # If raised_at is already after expiry_at (e.g., small window, or invalid data)
    # or total_steps is 0 (shouldn't happen with paths of length > 0),
    # handle gracefully to avoid division by zero or negative durations.
    if total_steps == 0 or total_duration.total_seconds() <= 0:
        # For a single step, or invalid dates, just use raised_at or expiry_at
        return raised_dt + timedelta(minutes=step) # Add small increment for multiple steps

    # Calculate the base interval for each step
    # We want to distribute the steps evenly between raised_at and expiry_at
    base_interval = total_duration / total_steps

    # Calculate the base time for the current step
    base_time = raised_dt + base_interval * step

    # Add a small random jitter to make it more realistic, but keep it within bounds
    # Jitter is a small percentage of the base_interval
    jitter_seconds = base_interval.total_seconds() * 0.1
    jitter = timedelta(seconds=random.uniform(-jitter_seconds, jitter_seconds))

    updated_at = base_time + jitter

    # Ensure the timestamp doesn't go before raised_at or after expiry_at due to jitter
    updated_at = max(raised_dt, min(updated_at, expiry_dt))

    return updated_at


def generate_ticket_history(tickets):
    STATUS_PATHS = {
        'a': [1, 2, 3, 4],
        'b': [1, 2, 5],
        'c': [1, 2, 3, 6, 2, 3, 4],
        'd': [1, 2, 3, 6, 2, 3, 5],
        'e': [1, 2, 7],
        'f': [1, 2, 7, 3, 4]
    }
    STATUS_ID_TO_NAME = {
        1: "Open", 2: "In Progress", 3: "Resolved",
        4: "Closed", 5: "Invalid", 6: "Escalated", 7: "Temp Resolved"
    }

    all_employees = list(set(tickets['EMPLOYEE_ID_FK']).union(set(tickets['ASSIGNED_TO'])))
    history_records = []

    for idx, ticket in tickets.iterrows():
        # Choose a status path based on weights
        paths, weights = zip(*[(v, w) for v, w in zip(STATUS_PATHS.values(), [70,3,7,5,7,8])])
        chosen_path = random.choices(paths, weights=weights, k=1)[0]

        assigned_to = ticket['ASSIGNED_TO']
        total_steps = len(chosen_path)

        for step, status_id in enumerate(chosen_path):
            # Update assigned_to and updated_by based on status
            if status_id in [6, 7]:  # Escalated or Temp Resolved
                # Change assignee on escalation or temp resolved
                possible_assignees = [emp for emp in all_employees if emp != assigned_to]
                assigned_to = random.choice(possible_assignees) if possible_assignees else assigned_to
                updated_by = assigned_to
            else:
                # Logic for other statuses
                if status_id == 1:  # Open
                    assigned_to = 0  # unassigned
                    updated_by = ticket['RAISED_BY']
                elif status_id == 2:  # In Progress
                    assigned_to = ticket['ASSIGNED_TO']
                    updated_by = assigned_to if random.random() < 0.8 else random.choice(all_employees)
                elif status_id == 3:  # Resolved
                    assigned_to = ticket['ASSIGNED_TO']
                    updated_by = assigned_to
                elif status_id == 4:  # Closed
                    assigned_to = ticket['ASSIGNED_TO']
                    updated_by = ticket['RAISED_BY']
                elif status_id == 5:  # Invalid
                    assigned_to = ticket['ASSIGNED_TO']
                    updated_by = assigned_to

            updated_at = generate_updated_at(ticket['RAISED_AT'], ticket['EXPIRY_AT'], step, total_steps)

            record = {
                'TICKET_ID': ticket['TICKET_ID '],
                'EMPLOYEE_ID_FK': ticket['EMPLOYEE_ID_FK'],
                'RAISED_AT': ticket['RAISED_AT'],
                'RAISED_BY': ticket['RAISED_BY'],
                'CATEGORY_ID': ticket['CATEGORY_ID'],
                'SUB_CATEGORY_ID': ticket['SUB_CATEGORY_ID'],
                'EXPIRY_AT': ticket['EXPIRY_AT'],
                'ASSIGNED_TO': assigned_to,
                'STATUS_ID': status_id,
                'STATUS_NAME': STATUS_ID_TO_NAME[status_id],
                'IS_ACTIVE': ticket['IS_ACTIVE'],
                'UPDATED_AT': updated_at,
                'UPDATED_BY': updated_by
            }
            history_records.append(record)

    # Sort by TICKET_ID and UPDATED_AT to ensure order
    history_df = pd.DataFrame(history_records)
    history_df.sort_values(['TICKET_ID', 'UPDATED_AT'], inplace=True)
    return history_df






def main():
    print("Please upload your input Excel file (e.g., tickets_input.xlsx):")
    #uploaded = files.upload("/content/Help desk data.xlsx")  # Upload widget

    # Get the uploaded filename
    #input_filename = list(uploaded.keys())[0]
    #print(f"Uploaded file: {input_filename}")

    # Read input Excel
    tickets = pd.read_excel("/content/Help desk data (1).xlsx",sheet_name = "Sheet1")
    tickets['temp_expiry_at'] = tickets['EXPIRY_AT']

    def maybe_extend_expiry(row):
      if random.random() < 0.17:
          extension_days = math.ceil(random.uniform(0.2, 5))
          return pd.to_datetime(row['EXPIRY_AT']) + timedelta(days=extension_days)
      else:
          return row['EXPIRY_AT']

    tickets['EXPIRY_AT'] = tickets.apply(maybe_extend_expiry, axis=1)


    #print(tickets.head(10))

    diff_mask = tickets[tickets['temp_expiry_at'] != tickets['EXPIRY_AT']]
    print(len(diff_mask))

    #print(diff_mask.head(10))
    # Generate history
    history_df = generate_ticket_history(tickets)
    print(history_df.head(10))
    print(len(history_df))

    history_df['EXPIRY_AT'] = history_df['TICKET_ID'].map(tickets.set_index('TICKET_ID ')['temp_expiry_at'])
    # Save output Excel
    output_filename = 'ticket_history_output.xlsx'
    history_df.to_excel(output_filename, index=False)
    print(f"Ticket history generated and saved to {output_filename}")

    # Provide download link
    #files.download(output_filename)

if __name__ == "__main__":
    main()

Please upload your input Excel file (e.g., tickets_input.xlsx):
349
   TICKET_ID  EMPLOYEE_ID_FK  RAISED_AT  RAISED_BY  CATEGORY_ID  \
0          1              22 2022-08-13         22            3   
1          1              22 2022-08-13         22            3   
2          1              22 2022-08-13         22            3   
3          2              17 2019-08-23         17            2   
4          2              17 2019-08-23         17            2   
5          2              17 2019-08-23         17            2   
6          2              17 2019-08-23         17            2   
7          3              46 2024-12-16         46            1   
8          3              46 2024-12-16         46            1   
9          3              46 2024-12-16         46            1   

   SUB_CATEGORY_ID  EXPIRY_AT  ASSIGNED_TO  STATUS_ID    STATUS_NAME  \
0                6 2022-08-21            0          1           Open   
1                6 2022-08-21           79        

In [ ]:
import pandas as pd
import random
from datetime import timedelta
import math

def generate_updated_at(raised_at, expiry_at, step, total_steps):
    raised_dt = pd.to_datetime(raised_at)
    expiry_dt = pd.to_datetime(expiry_at)
    total_duration = expiry_dt - raised_dt

    if total_steps == 0 or total_duration.total_seconds() <= 0:
        return raised_dt + timedelta(minutes=step)

    base_interval = total_duration / total_steps
    base_time = raised_dt + base_interval * step
    jitter_seconds = base_interval.total_seconds() * 0.1
    jitter = timedelta(seconds=random.uniform(-jitter_seconds, jitter_seconds))
    updated_at = base_time + jitter

    return max(raised_dt, min(updated_at, expiry_dt))


def generate_ticket_history(tickets):
    STATUS_PATHS = {
        'a': [1, 2, 3, 4],
        'b': [1, 2, 5],
        'c': [1, 2, 3, 6, 2, 3, 4],
        'd': [1, 2, 3, 6, 2, 5],
        'e': [1, 2, 7],
        'f': [1, 2, 7, 3, 4]
    }

    STATUS_ID_TO_NAME = {
        1: "Open", 2: "In Progress", 3: "Resolved",
        4: "Closed", 5: "Invalid", 6: "Escalated", 7: "Temp Resolved"
    }

    all_employees = list(set(tickets['EMPLOYEE_ID_FK']).union(set(tickets['ASSIGNED_TO'])))
    history_records = []

    for idx, ticket in tickets.iterrows():
        paths, weights = zip(*[(v, w) for v, w in zip(STATUS_PATHS.values(), [70, 3, 7, 5, 7, 8])])
        chosen_path = random.choices(paths, weights=weights, k=1)[0]

        assigned_to = ticket['ASSIGNED_TO']
        total_steps = len(chosen_path)
        escalated = False
        new_assignee = assigned_to
        raised_by = ticket['RAISED_BY']


        for step, status_id in enumerate(chosen_path):
            if escalated:
                assigned_to = new_assignee

            if status_id == 6:
                escalated = True
                updated_by = raised_by
            elif status_id == 7:
                updated_by = assigned_to
            else:
                if not escalated:
                    if status_id == 1:
                        assigned_to = 0
                        updated_by = ticket['RAISED_BY']
                    elif status_id == 2:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = assigned_to if random.random() < 0.8 else random.choice(all_employees)
                    elif status_id == 3:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = assigned_to
                    elif status_id == 4:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = ticket['RAISED_BY']
                    elif status_id == 5:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = assigned_to
                elif status_id == 4:
                    updated_by= raised_by
                else:
                    updated_by = assigned_to

            if escalated and status_id == 6:
                possible_assignees = [emp for emp in all_employees if emp != assigned_to]
                new_assignee = random.choice(possible_assignees) if possible_assignees else assigned_to

            updated_at = generate_updated_at(ticket['RAISED_AT'], ticket['EXPIRY_AT'], step, total_steps)

            record = {
                'TICKET_ID': ticket['TICKET_ID '],
                'EMPLOYEE_ID_FK': ticket['EMPLOYEE_ID_FK'],
                'RAISED_AT': ticket['RAISED_AT'],
                'RAISED_BY': ticket['RAISED_BY'],
                'CATEGORY_ID': ticket['CATEGORY_ID'],
                'SUB_CATEGORY_ID': ticket['SUB_CATEGORY_ID'],
                'EXPIRY_AT': ticket['EXPIRY_AT'],
                'ASSIGNED_TO': assigned_to,
                'STATUS_ID': status_id,
                'STATUS_NAME': STATUS_ID_TO_NAME[status_id],
                'IS_ACTIVE': ticket['IS_ACTIVE'],
                'UPDATED_AT': updated_at,
                'UPDATED_BY': updated_by
            }
            history_records.append(record)

    history_df = pd.DataFrame(history_records)
    history_df.sort_values(['TICKET_ID', 'UPDATED_AT'], inplace=True)
    return history_df


def maybe_extend_expiry(row):
    if random.random() < 0.17:
        extension_days = math.ceil(random.uniform(0.2, 5))
        return pd.to_datetime(row['EXPIRY_AT']) + timedelta(days=extension_days)
    return row['EXPIRY_AT']


def main():
    print("Processing ticket history generation...")

    input_path = "/content/Help desk data (1).xlsx"
    tickets = pd.read_excel(input_path, sheet_name="Sheet1")
    tickets['temp_expiry_at'] = tickets['EXPIRY_AT']
    tickets['EXPIRY_AT'] = tickets.apply(maybe_extend_expiry, axis=1)

    modified_expiries = tickets[tickets['temp_expiry_at'] != tickets['EXPIRY_AT']]
    print(f"Tickets with extended expiry: {len(modified_expiries)}")

    history_df = generate_ticket_history(tickets)
    print(history_df.head(10))
    print(f"Total history records: {len(history_df)}")

    # Fix column name if extra space exists
    tickets.columns = tickets.columns.str.strip()
    history_df['EXPIRY_AT'] = history_df['TICKET_ID'].map(tickets.set_index('TICKET_ID')['temp_expiry_at'])

    output_path = "ticket_history_output.xlsx"
    history_df.to_excel(output_path, index=False)
    print(f"Ticket history generated and saved to {output_path}")


if __name__ == "__main__":
    main()


Processing ticket history generation...
Tickets with extended expiry: 354
   TICKET_ID  EMPLOYEE_ID_FK  RAISED_AT  RAISED_BY  CATEGORY_ID  \
0          1              22 2022-08-13         22            3   
1          1              22 2022-08-13         22            3   
2          1              22 2022-08-13         22            3   
3          1              22 2022-08-13         22            3   
4          2              17 2019-08-23         17            2   
5          2              17 2019-08-23         17            2   
6          2              17 2019-08-23         17            2   
7          2              17 2019-08-23         17            2   
8          3              46 2024-12-16         46            1   
9          3              46 2024-12-16         46            1   

   SUB_CATEGORY_ID  EXPIRY_AT  ASSIGNED_TO  STATUS_ID  STATUS_NAME  IS_ACTIVE  \
0                6 2022-08-21            0          1         Open        NaN   
1                6 2022-08

In [ ]:
import pandas as pd
import random
from datetime import timedelta
import math

def generate_updated_at(raised_at, expiry_at, step, total_steps):
    raised_dt = pd.to_datetime(raised_at)
    expiry_dt = pd.to_datetime(expiry_at)
    total_duration = expiry_dt - raised_dt

    if total_steps == 0 or total_duration.total_seconds() <= 0:
        return raised_dt + timedelta(minutes=step)

    base_interval = total_duration / total_steps
    base_time = raised_dt + base_interval * step
    jitter_seconds = base_interval.total_seconds() * 0.1
    jitter = timedelta(seconds=random.uniform(-jitter_seconds, jitter_seconds))
    updated_at = base_time + jitter

    return max(raised_dt, min(updated_at, expiry_dt))


def get_higher_level_assignee(current_assignee, category_id, employee_map):
    current_level_series = employee_map.loc[
        (employee_map['ID'] == current_assignee) &
        (employee_map['CATEGORY_ID'] == category_id),
        'LEVEL_ID_FK'
    ]

    if current_level_series.empty:
        current_level = -1  # fallback level if not found
    else:
        current_level = current_level_series.values[0]

    higher_level_employees = employee_map.loc[
        (employee_map['CATEGORY_ID'] == category_id) &
        (employee_map['LEVEL_ID_FK'] > current_level),
        'ID'
    ].tolist()

    if higher_level_employees:
        return random.choice(higher_level_employees)
    else:
        return current_assignee  # fallback to current assignee if none found


def generate_ticket_history(tickets, employee_map):
    STATUS_PATHS = {
        'a': [1, 2, 3, 4],
        'b': [1, 2, 5],
        'c': [1, 2, 3, 6, 2, 3, 4],
        'd': [1, 2, 3, 6, 2, 5],
        'e': [1, 2, 7],
        'f': [1, 2, 7, 3, 4]
    }

    STATUS_ID_TO_NAME = {
        1: "Open", 2: "In Progress", 3: "Resolved",
        4: "Closed", 5: "Invalid", 6: "Escalated", 7: "Temp Resolved"
    }

    all_employees = list(set(tickets['EMPLOYEE_ID_FK']).union(set(tickets['ASSIGNED_TO'])))
    history_records = []

    for idx, ticket in tickets.iterrows():
        paths, weights = zip(*[(v, w) for v, w in zip(STATUS_PATHS.values(), [70, 3, 7, 5, 7, 8])])
        chosen_path = random.choices(paths, weights=weights, k=1)[0]

        assigned_to = ticket['ASSIGNED_TO']
        total_steps = len(chosen_path)
        escalated = False
        new_assignee = assigned_to
        raised_by = ticket['RAISED_BY']

        for step, status_id in enumerate(chosen_path):
            if escalated:
                assigned_to = new_assignee

            if status_id == 6:
                escalated = True
                # Get new assignee at higher level same category
                new_assignee = get_higher_level_assignee(assigned_to, ticket['CATEGORY_ID'], employee_map)
                updated_by = raised_by  # or assigned_to if you want
            elif status_id == 7:
                updated_by = assigned_to
            else:
                if not escalated:
                    if status_id == 1:
                        assigned_to = 0
                        updated_by = ticket['RAISED_BY']
                    elif status_id == 2:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = assigned_to if random.random() < 0.9 else get_higher_level_assignee(assigned_to, ticket['CATEGORY_ID'], employee_map)
                    elif status_id == 3:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = assigned_to
                    elif status_id == 4:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = ticket['RAISED_BY']
                    elif status_id == 5:
                        assigned_to = ticket['ASSIGNED_TO']
                        updated_by = assigned_to
                elif status_id == 4:
                    updated_by = raised_by
                else:
                    updated_by = assigned_to

            updated_at = generate_updated_at(ticket['RAISED_AT'], ticket['EXPIRY_AT'], step, total_steps)

            record = {
                'TICKET_ID': ticket['TICKET_ID '],
                'EMPLOYEE_ID_FK': ticket['EMPLOYEE_ID_FK'],
                'RAISED_AT': ticket['RAISED_AT'],
                'RAISED_BY': ticket['RAISED_BY'],
                'CATEGORY_ID': ticket['CATEGORY_ID'],
                'SUB_CATEGORY_ID': ticket['SUB_CATEGORY_ID'],
                'EXPIRY_AT': ticket['EXPIRY_AT'],
                'ASSIGNED_TO': assigned_to,
                'STATUS_ID': status_id,
                'STATUS_NAME': STATUS_ID_TO_NAME[status_id],
                'IS_ACTIVE': ticket['IS_ACTIVE'],
                'UPDATED_AT': updated_at,
                'UPDATED_BY': updated_by
            }
            history_records.append(record)

    history_df = pd.DataFrame(history_records)
    history_df.sort_values(['TICKET_ID', 'UPDATED_AT'], inplace=True)
    return history_df


def maybe_extend_expiry(row):
    if random.random() < 0.17:
        extension_days = math.ceil(random.uniform(0.2, 5))
        return pd.to_datetime(row['EXPIRY_AT']) + timedelta(days=extension_days)
    return row['EXPIRY_AT']


def main():
    print("Processing ticket history generation...")

    input_path = "/content/Help desk data (1).xlsx"
    tickets = pd.read_excel(input_path, sheet_name="Sheet1")
    employee_map = pd.read_excel(input_path, sheet_name="employee_cat_level_map")

    tickets['temp_expiry_at'] = tickets['EXPIRY_AT']
    tickets['EXPIRY_AT'] = tickets.apply(maybe_extend_expiry, axis=1)

    modified_expiries = tickets[tickets['temp_expiry_at'] != tickets['EXPIRY_AT']]
    print(f"Tickets with extended expiry: {len(modified_expiries)}")

    history_df = generate_ticket_history(tickets, employee_map)
    print(history_df.head(10))
    print(f"Total history records: {len(history_df)}")

    tickets.columns = tickets.columns.str.strip()
    history_df['EXPIRY_AT'] = history_df['TICKET_ID'].map(tickets.set_index('TICKET_ID')['temp_expiry_at'])

    output_path = "ticket_history_output.xlsx"
    history_df.to_excel(output_path, index=False)
    print(f"Ticket history generated and saved to {output_path}")


if __name__ == "__main__":
    main()


Processing ticket history generation...
Tickets with extended expiry: 338
   TICKET_ID  EMPLOYEE_ID_FK  RAISED_AT  RAISED_BY  CATEGORY_ID  \
0          1              22 2022-08-13         22            3   
1          1              22 2022-08-13         22            3   
2          1              22 2022-08-13         22            3   
3          1              22 2022-08-13         22            3   
4          1              22 2022-08-13         22            3   
5          1              22 2022-08-13         22            3   
6          1              22 2022-08-13         22            3   
7          2              17 2019-08-23         17            2   
8          2              17 2019-08-23         17            2   
9          2              17 2019-08-23         17            2   

   SUB_CATEGORY_ID  EXPIRY_AT  ASSIGNED_TO  STATUS_ID  STATUS_NAME  IS_ACTIVE  \
0                6 2022-08-21            0          1         Open        NaN   
1                6 2022-08

In [ ]:
import math
for i in range(1,10):
  print(math.ceil(random.uniform(0.2, 3)))

3
2
1
1
3
3
3
2
2


In [ ]:
df1 = pd.read_excel("/content/ticket_history_output.xlsx")

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
result_df = pd.DataFrame()

for i in range(1, 1000):
    filtered_rows = df1[(df1['EXPIRY_AT'] < df1['UPDATED_AT']) & (df1['TICKET_ID'] == i)]
    if not filtered_rows.empty:
        result_df = pd.concat([result_df, filtered_rows])

print(result_df)


      TICKET_ID  EMPLOYEE_ID_FK  RAISED_AT  RAISED_BY  CATEGORY_ID  \
21            4             169 2025-06-10        169            1   
40            9              99 2024-01-12         99            4   
52           12             174 2021-09-13        174            1   
75           17              83 2024-09-01         83            4   
159          36             182 2024-07-12        182            3   
...         ...             ...        ...        ...          ...   
4155        975             151 2025-04-12        151            2   
4163        977              33 2025-06-03         33            3   
4184        982             102 2024-03-09        102            2   
4195        985              88 2024-11-02         88            3   
4240        996             190 2024-02-08         68            1   

      SUB_CATEGORY_ID  EXPIRY_AT  ASSIGNED_TO  STATUS_ID    STATUS_NAME  \
21                  1 2025-06-19          117          4         Closed   
40       

In [ ]:
df1[df1['TICKET_ID'] == 3]

,TICKET_ID,EMPLOYEE_ID_FK,RAISED_AT,RAISED_BY,CATEGORY_ID,SUB_CATEGORY_ID,EXPIRY_AT,ASSIGNED_TO,STATUS_ID,STATUS_NAME,IS_ACTIVE,UPDATED_AT,UPDATED_BY
11,3,46,2024-12-16,46,1,1,2024-12-19,0,1,Open,NaN,2024-12-16 00:00:00.000,46
12,3,46,2024-12-16,46,1,1,2024-12-19,11,2,In Progress,NaN,2024-12-16 10:00:14.532,11
13,3,46,2024-12-16,46,1,1,2024-12-19,11,3,Resolved,NaN,2024-12-16 21:14:04.755,11
14,3,46,2024-12-16,46,1,1,2024-12-19,11,6,Escalated,NaN,2024-12-17 05:56:07.016,46
15,3,46,2024-12-16,46,1,1,2024-12-19,2,2,In Progress,NaN,2024-12-17 16:24:56.588,2
16,3,46,2024-12-16,46,1,1,2024-12-19,2,3,Resolved,NaN,2024-12-18 03:14:43.340,2
17,3,46,2024-12-16,46,1,1,2024-12-19,2,4,Closed,NaN,2024-12-18 13:51:42.545,46


In [ ]:
df1[df1['TICKET_ID'] == 13]


,TICKET_ID,EMPLOYEE_ID_FK,RAISED_AT,RAISED_BY,CATEGORY_ID,SUB_CATEGORY_ID,EXPIRY_AT,ASSIGNED_TO,STATUS_ID,STATUS_NAME,IS_ACTIVE,UPDATED_AT,UPDATED_BY
53,13,14,2025-03-10,14,2,3,2025-03-15,0,1,Open,NaN,2025-03-10 00:14:32.590,14
54,13,14,2025-03-10,14,2,3,2025-03-15,201,2,In Progress,NaN,2025-03-11 02:17:06.197,201
55,13,14,2025-03-10,14,2,3,2025-03-15,201,7,Temp Resolved,NaN,2025-03-11 21:37:33.910,201
56,13,14,2025-03-10,14,2,3,2025-03-15,201,3,Resolved,NaN,2025-03-13 01:43:10.878,201
57,13,14,2025-03-10,14,2,3,2025-03-15,201,4,Closed,NaN,2025-03-13 23:54:17.604,14


In [ ]:
df1[df1['STATUS_ID'] ==6]

,TICKET_ID,EMPLOYEE_ID_FK,RAISED_AT,RAISED_BY,CATEGORY_ID,SUB_CATEGORY_ID,EXPIRY_AT,ASSIGNED_TO,STATUS_ID,STATUS_NAME,IS_ACTIVE,UPDATED_AT,UPDATED_BY
3,1,22,2022-08-13,22,3,6,2022-08-21,79,6,Escalated,NaN,2022-08-16 10:30:45.501,22
14,3,46,2024-12-16,46,1,1,2024-12-19,11,6,Escalated,NaN,2024-12-17 05:56:07.016,46
68,16,105,2023-04-19,105,3,6,2023-04-24,132,6,Escalated,NaN,2023-04-21 01:43:51.281,105
87,20,40,2022-04-17,127,1,1,2022-04-21,84,6,Escalated,NaN,2022-04-18 16:29:43.954,127
119,27,112,2022-03-01,112,3,5,2022-03-05,10,6,Escalated,NaN,2022-03-02 16:52:08.454,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,1995,187,2025-05-25,187,2,3,2025-06-04,189,6,Escalated,NaN,2025-05-29 04:00:03.814,187
8565,1996,77,2023-11-19,77,3,5,2023-11-26,177,6,Escalated,NaN,2023-11-22 10:18:26.725,77
8587,2001,93,2021-09-02,93,3,6,2021-09-05,29,6,Escalated,NaN,2021-09-05 11:08:08.733,93
8598,2003,14,2021-12-10,14,2,3,2021-12-12,96,6,Escalated,NaN,2021-12-11 00:25:44.482,14


In [ ]:
df1[df1['TICKET_ID'] == 2031]

,TICKET_ID,EMPLOYEE_ID_FK,RAISED_AT,RAISED_BY,CATEGORY_ID,SUB_CATEGORY_ID,EXPIRY_AT,ASSIGNED_TO,STATUS_ID,STATUS_NAME,IS_ACTIVE,UPDATED_AT,UPDATED_BY
8725,2031,97,2021-05-01,97,2,3,2021-05-11,0,1,Open,NaN,2021-05-01 00:00:00.000,97
8726,2031,97,2021-05-01,97,2,3,2021-05-11,63,2,In Progress,NaN,2021-05-02 12:15:01.160,43
8727,2031,97,2021-05-01,97,2,3,2021-05-11,63,3,Resolved,NaN,2021-05-04 05:34:50.323,63
8728,2031,97,2021-05-01,97,2,3,2021-05-11,63,6,Escalated,NaN,2021-05-06 01:31:06.282,97
8729,2031,97,2021-05-01,97,2,3,2021-05-11,96,2,In Progress,NaN,2021-05-07 12:37:29.054,96
8730,2031,97,2021-05-01,97,2,3,2021-05-11,96,5,Invalid,NaN,2021-05-09 09:38:04.134,96
